# Laboratorio 2: Data Understanding

**Universidad del Valle de Guatemala**  
**Facultad de Ingeniería**  
**Departamento de Ciencias de la Computación**  
**Machine Learning Operations** 

## Integrantes

- Arturo Argueta - 21527 
- Edwin de León - 22809 
- Diego Leiva - 21752 
- Pablo Orellana - 21970

## Librerías

In [1]:
from pathlib import Path
import time
import pandas as pd
from tqdm import tqdm

## Lectura de datos

Durante el análisis exploratorio se encontró que hay conjuntos de datos con codificaciones diferentes a la típica utf-8, por lo que se necesita una lectura segura de datos

In [2]:
def read_csv_with_fallback(path, encodings=("utf-8", "latin1", "cp1252"), **pd_kwargs):
    """
    Intenta leer un CSV probando varias codificaciones para encontrar la correcta.

    Args:
        path (str): Ruta al archivo CSV.
        encodings (tuple): Codificaciones a probar.

    Returns:
        pd.DataFrame: DataFrame leído.
    """
    last_err = None

    # Itera sobre las codificaciones
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc, **pd_kwargs)
            return df, enc
        except UnicodeDecodeError as e:
            last_err = e
        except Exception as e:
            # Otros errores; guarda y sigue probando por si es solo encoding
            last_err = e

    # Último intento “tolerante”: utf-8 con reemplazo para caracteres malos
    try:
        with open(path, "r", encoding="utf-8", errors="replace") as f:
            df = pd.read_csv(f, **pd_kwargs)
        return df, "utf-8(errors=replace)"
    except Exception:
        raise last_err

In [ ]:
def load_csvs(
    folder: str,
    pattern: str = "*.csv",
    encodings=("utf-8", "latin1", "cp1252"),
    **pd_kwargs
):
    """
    Lee todos los CSVs que hagan match con `pattern` en `folder`,
    mostrando progreso con tqdm y midiendo el tiempo por archivo.

    Args:
        folder (str): Carpeta donde buscar los archivos CSV.
        pattern (str): Patrón de búsqueda para los archivos CSV.
        encodings (tuple): Codificaciones a probar al leer los CSVs.
        **pd_kwargs: Argumentos adicionales para pd.read_csv.

    Returns:
        tuple: (dfs, report) donde dfs es un diccionario de DataFrames y report es un DataFrame con el resumen de la carga.
    """
    # Obtiene la lista de archivos CSV
    files = sorted(Path(folder).glob(pattern))
    dfs = {}
    rows = []

    # Itera sobre los archivos CSV
    for p in tqdm(files, desc="Leyendo CSVs", unit="archivo"):
        t0 = time.perf_counter()  # Marca el tiempo de inicio
        status = "ok"  # Estado inicial
        used_enc = None  # Codificación utilizada
        nrows = ncols = None  # Filas y columnas
        err_msg = ""  # Mensaje de error
        try:
            # Intenta leer el CSV
            df, used_enc = read_csv_with_fallback(p, encodings=encodings, **pd_kwargs)
            dfs[p.name] = df
            nrows, ncols = df.shape
        except Exception as e:
            # Si falla, captura el error
            status = "error"
            err_msg = f"{type(e).__name__}: {e}"
        t1 = time.perf_counter()
        rows.append(
            {
                "archivo": p.name,
                "estado": status,
                "encoding": used_enc,
                "filas": nrows,
                "columnas": ncols,
                "tiempo_s": round(t1 - t0, 3),
                "ruta": str(p),
                "error": err_msg,
            }
        )
    report = pd.DataFrame(rows).sort_values(["estado", "archivo"])
    return dfs, report

In [4]:
# Lee todos los CSV de la carpeta "data", separador por coma (ajusta sep si necesitas ; o \t)
dfs, reporte = load_csvs("data/raw", sep=",")
display(reporte)


Leyendo CSVs: 100%|██████████| 5/5 [00:00<00:00,  8.02archivo/s]


,archivo,estado,encoding,filas,columnas,tiempo_s,ruta,error
0,categoria.csv,ok,utf-8,101,2,0.002,data\raw\categoria.csv,
1,cliente.csv,ok,latin1,12000,12,0.035,data\raw\cliente.csv,
2,evento.csv,ok,utf-8,2756101,5,0.576,data\raw\evento.csv,
3,marca.csv,ok,utf-8,307,2,0.001,data\raw\marca.csv,
4,producto.csv,ok,utf-8,12026,6,0.008,data\raw\producto.csv,
